# Langchain Tests

In [6]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain


# Load environment variables
load_dotenv()

# Get environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY environment variable not set")


# Create the language model
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

# Create the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

# Create the output parser
output_parser = StrOutputParser()

# Create the chain
chain = prompt | llm | output_parser

# Create the document loader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

# Create the embeddings
embeddings = OpenAIEmbeddings()

# Create the vector store
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

# Create the document chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

from langchain_core.documents import Document

result = document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


LangSmith can help with testing by providing the ability to run chains over data points and visualize the outputs. It simplifies dataset uploading and allows for the review and evaluation of test runs. LangSmith also offers evaluators that can be specified during a test run to assess the results. Additionally, LangSmith facilitates human evaluation through annotation queues, allowing reviewers to manually assess and provide feedback on runs.
